In [1]:
# download minsearch
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-04 07:44:25--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0.002s  

2024-07-04 07:44:25 (2.13 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [3]:
# Import packages
import requests 
import minsearch
from openai import OpenAI

In [4]:
# setup minsearch index
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [5]:
# setup search function
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [6]:
# setup prompt
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [7]:
# setup llm
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [8]:
# setup rag
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [9]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [10]:
print(rag("I just discovered the course. Can I still join in?"))

 Based on the context from the course FAQ database:

Yes, if you just discovered the course and missed the start date of January 15th, 2024 at 17h00, there are still options available to join in. While students who register beforehand will have access to live sessions starting with "Office Hours," those unable or late to attend can submit homework assignments even after the course has started and continue taking final project submissions as long as they meet deadlines set by instructors (Note: Deadlines apply, so plan your work accordingly).

As for joining a self-paced mode of study post-course completion, all materials are retained and accessible. You can follow the online course at any time afterward to learn more about prerequisites or syllabus contents if needed (e.g., homework submission process, final capstone project preparations). There is also a support channel on DataTalks.Club's Slack where you may seek help during the self-paced mode of study; however, instructors advise u